In [ ]:
import scipy.io
import torch
import numpy as np
import torch.nn as nn
import torch.utils.data as Data
import matplotlib.pyplot as plt
import torch.nn.functional as F
#from tensorboardX import SummaryWriter
from sklearn.metrics import roc_auc_score,roc_curve,auc,average_precision_score,precision_recall_curve
torch.manual_seed(1337)
np.random.seed(1337)
torch.cuda.manual_seed(1337)
torch.backends.cudnn.benchmark=True

In [ ]:
print('starting loading the data')
np_test_data = scipy.io.loadmat('test.mat')

In [ ]:
y_test_Pol2 = np_test_data['testdata'][:, [332,354,408,431,432,433,451,453,455,457,459,461,463]].sum(axis=1) > 0

X_test_NRSF = np_test_data['testxdata'][y_test_Pol2]

In [ ]:
y_test_NRSF.shape

In [ ]:
X_test_NRSF.shape

In [ ]:
num=0
true_list=[]
for i in range(len(y_test_NRSF)):
    if(y_test_NRSF[i]==True):
        true_list.append(i)
        num+=1    

In [ ]:
num

In [ ]:
total_pos_train = np_test_data['testdata'].sum()
print('Sparsity: %0.4f' % (1 - 1.0 * total_pos_train / np.prod(np_test_data['testdata'].shape)))

In [ ]:
print('compling the network')
class DanQ(nn.Module):
    def __init__(self, ):
        super(DanQ, self).__init__()
        self.Conv1 = nn.Conv1d(in_channels=4, out_channels=320, kernel_size=13)
        #nn.init.uniform_(self.Conv1.weight, -0.05, 0.05)
        self.Maxpool = nn.MaxPool1d(kernel_size=13, stride=6)
        self.Drop1 = nn.Dropout(p=0.2)
        self.BiLSTM = nn.LSTM(input_size=320, hidden_size=320, num_layers=2,
                                 batch_first=True,
                                 dropout=0.5,
                                 bidirectional=True)
        self.Linear1 = nn.Linear(163*640, 925)
        self.Linear2 = nn.Linear(925, 919)


    def forward(self, input):
        x = self.Conv1(input)
        x1 = F.relu(x)
        x = self.Maxpool(x1)
        x = self.Drop1(x)
        x_x = torch.transpose(x, 1, 2)
        x, (h_n,h_c) = self.BiLSTM(x_x)
        #x, h_n = self.BiGRU(x_x)
        x = x.contiguous().view(-1, 163*640)
        x = self.Linear1(x)
        x = F.relu(x)
        x = self.Linear2(x)
        #x = torch.sigmoid(x)
        return x1,x

danq = DanQ()
danq.load_state_dict(torch.load('model/model0512_2/danq_net_params_4.pkl'))

In [ ]:
motifs = np.zeros((320, 4, 13))
nsites = np.zeros(320)
danq.eval()
for i in range(0, len(X_test_NRSF), 100):
    x = X_test_NRSF[i:i+100]
    x_tensor = torch.FloatTensor(x)
    #print(seq.shape)
    conv_output, _ = danq(x_tensor)
    max_inds = np.argmax(conv_output.cpu().detach().numpy().data, axis=2)
    max_acts = np.max(conv_output.cpu().detach().numpy().data, axis=2)
    #print(max_inds.shape)
    #print(max_acts.shape)
    for m in range(320):
        for n in range(len(x)):
            if max_acts[n, m] > 0:
                nsites[m] += 1
                motifs[m] += x[n, :, max_inds[n, m]:max_inds[n, m]+13]

In [ ]:
motifs.shape

In [ ]:
max_acts.shape

In [ ]:
conv_output.cpu().detach().numpy().data.shape

In [ ]:
num=0
for m in range(320):
        for n in range(len(x)):
            if max_acts[n, m] < 0:
                num+=1

In [ ]:
num

In [ ]:
motifs = np.transpose(motifs,(0, 2, 1))
print(motifs.shape)
print('Making motifs')

motifs = motifs[:, :, [0, 2, 1, 3]]

motifs_file = open('motifs_Pol2.txt', 'w')
motifs_file.write('MEME version 5.0.5\n\n'
                  'ALPHABET= ACGT\n\n'
                  'strands: + -\n\n'
                  'Background letter frequencies (from uniform background):\n'
                  'A 0.25000 C 0.25000 G 0.25000 T 0.25000\n\n')

for m in range(320):
    if nsites[m] == 0:
        continue
    motifs_file.write('MOTIF M%i O%i\n' % (m, m))
    motifs_file.write("letter-probability matrix: alength= 4 w= %i nsites= %i E= 1337.0e-6\n" % (13, nsites[m]))
    for j in range(13):
        motifs_file.write("%f %f %f %f\n" % tuple(1.0 * motifs[m, j, 0:4] / np.sum(motifs[m, j, 0:4])))
    motifs_file.write('\n')

motifs_file.close()


In [ ]:
motifs = np.transpose(motifs,(0, 2, 1))
print(motifs.shape)
print('Making motifs')

motifs = motifs[:, :, [0, 2, 1, 3]]

motifs_file = open('motifs_Pol2_heatmap.txt', 'w')
for m in range(320):
    if nsites[m] == 0:
        continue
    for j in range(13):
        p = 1.0 * motifs[m, j, 0:4] / np.sum(motifs[m, j, 0:4])
        motifs_file.write("%f %f %f %f\n" % tuple(1.0 * motifs[m, j, 0:4] / np.sum(motifs[m, j, 0:4])))
    motifs_file.write('\n')

motifs_file.close()